In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../summary_data/original/all_tools_TRB.csv")
df

Sample               CDR3  nReads_TCR  nReads_MIXCR  nReads_IMREP  \
0       SRR5233637   CASSPRVTSGTYEQYF        32.0           0.0           0.0   
1       SRR5233637    CASSYSDRGGQPQHF        13.0           0.0           0.0   
2       SRR5233637     CASKVALGGETQYF        25.0           0.0           0.0   
3       SRR5233637  CASRAPGTGTLGSPLHF        66.0           0.0           0.0   
4       SRR5233637   CASSSGQGGPSTEAFF        52.0           0.0           0.0   
...            ...                ...         ...           ...           ...   
375331    sample14    CASSESPAFGEKLFF         0.0           0.0           0.0   
375332    sample14     CASSWTGSQETQYF         0.0           0.0           0.0   
375333    sample14   CASRTGLAGGIGELFF         0.0           0.0           0.0   
375334    sample14       CASSVEGYEQYF         0.0           0.0           0.0   
375335    sample14    CASSQQGDSNQPQHF         0.0           0.0           0.0   

        nReads_TRUST4           tissue  tissue_type  total_reads_TCR  \
0                 0.0  small_intestine  T_cell_poor        3047629.0   
1                 0.0  small_intestine  T_cell_poor        3047629.0   
2                 0.0  small_intestine  T_cell_poor        3047629.0   
3                 0.0  small_intestine  T_cell_poor        3047629.0   
4                 0.0  small_intestine  T_cell_poor        3047629.0   
...               ...              ...          ...              ...   
375331            3.0         melanoma  T_cell_poor         749686.0   
375332            2.0         melanoma  T_cell_poor         749686.0   
375333            2.0         melanoma  T_cell_poor         749686.0   
375334            2.0         melanoma  T_cell_poor         749686.0   
375335            2.0         melanoma  T_cell_poor         749686.0   

        total_reads_MIXCR  total_reads_IMREP  total_reads_TRUST4  \
0                    84.0              315.0               544.0   
1                    84.0              315.0               544.0   
2                    84.0              315.0               544.0   
3                    84.0              315.0               544.0   
4                    84.0              315.0               544.0   
...                   ...                ...                 ...   
375331              113.0              152.0               351.0   
375332              113.0              152.0               351.0   
375333              113.0              152.0               351.0   
375334              113.0              152.0               351.0   
375335              113.0              152.0               351.0   

        frequency_TCR  frequency_MIXCR  frequency_IMREP  frequency_TRUST4  \
0            0.000010              0.0              0.0          0.000000   
1            0.000004              0.0              0.0          0.000000   
2            0.000008              0.0              0.0          0.000000   
3            0.000022              0.0              0.0          0.000000   
4            0.000017              0.0              0.0          0.000000   
...               ...              ...              ...               ...   
375331       0.000000              0.0              0.0          0.008547   
375332       0.000000              0.0              0.0          0.005698   
375333       0.000000              0.0              0.0          0.005698   
375334       0.000000              0.0              0.0          0.005698   
375335       0.000000              0.0              0.0          0.005698   

       repertoire_type                   class  
0           polyclonal  T_cell_poor_polyclonal  
1           polyclonal  T_cell_poor_polyclonal  
2           polyclonal  T_cell_poor_polyclonal  
3           polyclonal  T_cell_poor_polyclonal  
4           polyclonal  T_cell_poor_polyclonal  
...                ...                     ...  
375331      polyclonal  T_cell_poor_polyclonal  
375332      polyclonal  T_cell_poor_poly

In [3]:
RNA_seq_reads = pd.read_csv("../summary_data/original/RNA_seq_reads.csv")
RNA_seq_reads

Sample  total_reads_RNA_seq
0       sample01            104984482
1       sample02             73892845
2       sample03             71654976
3       sample04             43179989
4       sample05             40524817
5       sample06             82476159
6       sample07             72397468
7       sample08             85492431
8       sample09             68584414
9       sample10             63320771
10      sample11             55727841
11      sample12            107919183
12      sample13             80622502
13      sample14             55931661
14    SRR5233637             72720367
15    SRR5233639             65998918
16  TCGA-CZ-4862             58529923
17  TCGA-CZ-5463            122632451
18  TCGA-CZ-5985             83862519

Total number of reads in each sample

In [4]:
total_reads = df[['Sample','tissue','tissue_type','repertoire_type','class','total_reads_TCR','total_reads_MIXCR','total_reads_IMREP','total_reads_TRUST4']]
total_reads = total_reads.drop_duplicates(keep='first')
total_reads

# Add the column of RNA-Seq reads
total_reads = pd.merge(total_reads, RNA_seq_reads, how='left', on=['Sample'])
total_reads

# Calculate number of TCR derived reads per one million RNA-Seq reads 
# MIXCR
MIXCR = total_reads[['Sample','tissue','tissue_type','repertoire_type','class','total_reads_TCR','total_reads_MIXCR','total_reads_RNA_seq']]
MIXCR.rename(columns={'total_reads_MIXCR':'total_reads_tool'}, inplace=True)
MIXCR.loc[:,'tool'] = 'MIXCR'
MIXCR.loc[:,'TCR_derived_by_RNA_seq_tool'] = MIXCR['total_reads_tool']/MIXCR['total_reads_RNA_seq'] *1000000

# IMREP
IMREP = total_reads[['Sample','tissue','tissue_type','repertoire_type','class','total_reads_TCR','total_reads_IMREP','total_reads_RNA_seq']]
IMREP.rename(columns={'total_reads_IMREP':'total_reads_tool'}, inplace=True)
IMREP.loc[:,'tool'] = 'IMREP'
IMREP.loc[:,'TCR_derived_by_RNA_seq_tool'] = IMREP['total_reads_tool']/IMREP['total_reads_RNA_seq']*1000000

# TRUST4
TRUST4 = total_reads[['Sample','tissue','tissue_type','repertoire_type','class','total_reads_TCR','total_reads_TRUST4','total_reads_RNA_seq']]
TRUST4.rename(columns={'total_reads_TRUST4':'total_reads_tool'}, inplace=True)
TRUST4.loc[:,'tool'] = 'TRUST4'
TRUST4.loc[:,'TCR_derived_by_RNA_seq_tool'] = TRUST4['total_reads_tool']/TRUST4['total_reads_RNA_seq']*1000000

reads_count = pd.concat([MIXCR,IMREP,TRUST4])
reads_count.to_csv('../summary_data/original/all_tools_TRB_reads.csv', index=False)

reads_count

/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

Sample           tissue  tissue_type repertoire_type  \
0     SRR5233637  small_intestine  T_cell_poor      polyclonal   
1     SRR5233639       lymph_node  T_cell_rich      polyclonal   
2   TCGA-CZ-4862           kidney  T_cell_poor      polyclonal   
3   TCGA-CZ-5463           kidney  T_cell_poor      polyclonal   
4   TCGA-CZ-5985           kidney  T_cell_poor      polyclonal   
5       sample01             PBMC  T_cell_rich      monoclonal   
6       sample02             PBMC  T_cell_rich      monoclonal   
7       sample03             PBMC  T_cell_rich      monoclonal   
8       sample04             PBMC  T_cell_rich      polyclonal   
9       sample05             PBMC  T_cell_rich      polyclonal   
10      sample06         melanoma  T_cell_poor      polyclonal   
11      sample07         melanoma  T_cell_poor      polyclonal   
12      sample08         melanoma  T_cell_poor      polyclonal   
13      sample09         melanoma  T_cell_poor      polyclonal   
14      sample10         melanoma  T_cell_poor      polyclonal   
15      sample11         melanoma  T_cell_poor      polyclonal   
16      sample12         melanoma  T_cell_poor      polyclonal   
17      sample13         melanoma  T_cell_poor      monoclonal   
18      sample14         melanoma  T_cell_poor      polyclonal   
0     SRR5233637  small_intestine  T_cell_poor      polyclonal   
1     SRR5233639       lymph_node  T_cell_rich      polyclonal   
2   TCGA-CZ-4862           kidney  T_cell_poor      polyclonal   
3   TCGA-CZ-5463           kidney  T_cell_poor      polyclonal   
4   TCGA-CZ-5985           kidney  T_cell_poor      polyclonal   
5       sample01             PBMC  T_cell_rich      monoclonal   
6       sample02             PBMC  T_cell_rich      monoclonal   
7       sample03             PBMC  T_cell_rich      monoclonal   
8       sample04             PBMC  T_cell_rich      polyclonal   
9       sample05             PBMC  T_cell_rich      polyclonal   
10      sample06         melanoma  T_cell_poor      polyclonal   
11      sample07         melanoma  T_cell_poor      polyclonal   
12      sample08         melanoma  T_cell_poor      polyclonal   
13      sample09         melanoma  T_cell_poor      polyclonal   
14      sample10         melanoma  T_cell_poor      polyclonal   
15      sample11         melanoma  T_cell_poor      polyclonal   
16      sample12         melanoma  T_cell_poor      polyclonal   
17      sample13         melanoma  T_cell_poor      monoclonal   
18      sample14         melanoma  T_cell_poor      polyclonal   
0     SRR5233637  small_intestine  T_cell_poor      polyclonal   
1     SRR5233639       lymph_node  T_cell_rich      polyclonal   
2   TCGA-CZ-4862           kidney  T_cell_poor      polyclonal   
3   TCGA-CZ-5463           kidney  T_cell_poor      polyclonal   
4   TCGA-CZ-5985           kidney  T_cell_poor      polyclonal   
5       sample01             PBMC  T_cell_rich      monoclonal   
6       sample02             PBMC  T_cell_rich      monoclonal   
7       sample03             PBMC  T_cell_rich      monoclonal   
8       sample04             PBMC  T_cell_rich      polyclonal   
9       sample05             PBMC  T_cell_rich      polyclonal   
10      sample06         melanoma  T_cell_poor      polyclonal   
11      sample07         melanoma  T_cell_poor      polyclonal   
12      sample08         melanoma  T_cell_poor      polyclonal   
13      sample09         melanoma  T_cell_poor      polyclonal   
14      sample10         melanoma  T_cell_poor      polyclonal   
15      sample11         melanoma  T_cell_poor      polyclonal   
16      sample12         melanoma  T_cell_poor      polyclonal   
17      sample13         melanoma  T_cell_poor      monoclonal   
18      sample14         melanoma  T_cell_poor      polyclonal   

                     class  total_reads_TCR  total_reads_tool  \
0   T_cell_poor_polyclonal        3047629.0              84.0   
1   T_cell_rich_polyclonal        3256697.0        

Average number of reads in different tissue types

In [5]:
mean_tissue_reads = reads_count.groupby("tissue")["total_reads_TCR","total_reads_tool","total_reads_RNA_seq","TCR_derived_by_RNA_seq_tool"].agg(["mean", "std"]) 
display(mean_tissue_reads)
tools = ['MIXCR','IMREP','TRUST4']              
for tool in tools: 
    print(tool)
    df_tool = reads_count.loc[reads_count['tool'] == tool]
    mean_tissue_reads = df_tool.groupby("tissue")["total_reads_TCR","total_reads_tool","total_reads_RNA_seq","TCR_derived_by_RNA_seq_tool"].agg(["mean", "std"]) 
    display(mean_tissue_reads)

total_reads_TCR                total_reads_tool                \
                           mean            std             mean           std   
tissue                                                                          
PBMC               1.215376e+05  100337.171124     68408.733333  82619.887336   
kidney             1.019600e+04    7230.794770       160.777778    247.037334   
lymph_node         3.256697e+06       0.000000      5514.333333   3892.265707   
melanoma           9.512597e+05  523470.109037       360.592593    590.024723   
small_intestine    3.047629e+06       0.000000       314.333333    230.000725   

                total_reads_RNA_seq               TCR_derived_by_RNA_seq_tool  \
                               mean           std                        mean   
tissue                                                                          
PBMC                   6.684742e+07  2.440819e+07                  769.672661   
kidney                 8.834163e+07  2.795975e+07                    2.192968   
lymph_node             6.599892e+07  0.000000e+00                   83.551875   
melanoma               7.471916e+07  1.593224e+07                    4.763433   
small_intestine        7.272037e+07  0.000000e+00                    4.322494   

                             
                        std  
tissue                       
PBMC             807.317795  
kidney             3.951345  
lymph_node        58.974690  
melanoma           7.380718  
small_intestine    3.162810

MIXCR


total_reads_TCR                total_reads_tool                \
                           mean            std             mean           std   
tissue                                                                          
PBMC               1.215376e+05  108376.531412     45224.200000  54187.320244   
kidney             1.019600e+04    8349.402613         0.000000      0.000000   
lymph_node         3.256697e+06            NaN      1346.000000           NaN   
melanoma           9.512597e+05  544844.963860       201.555556    339.992320   
small_intestine    3.047629e+06            NaN        84.000000           NaN   

                total_reads_RNA_seq               TCR_derived_by_RNA_seq_tool  \
                               mean           std                        mean   
tissue                                                                          
PBMC                   6.684742e+07  2.636386e+07                  510.665299   
kidney                 8.834163e+07  3.228514e+07                    0.000000   
lymph_node             6.599892e+07           NaN                   20.394274   
melanoma               7.471916e+07  1.658280e+07                    2.655950   
small_intestine        7.272037e+07           NaN                    1.155110   

                             
                        std  
tissue                       
PBMC             531.447948  
kidney             0.000000  
lymph_node              NaN  
melanoma           4.223584  
small_intestine         NaN

IMREP


total_reads_TCR                total_reads_tool                \
                           mean            std             mean           std   
tissue                                                                          
PBMC               1.215376e+05  108376.531412     68652.800000  89472.052227   
kidney             1.019600e+04    8349.402613        29.666667     17.953644   
lymph_node         3.256697e+06            NaN      6143.000000           NaN   
melanoma           9.512597e+05  544844.963860       214.888889    264.090157   
small_intestine    3.047629e+06            NaN       315.000000           NaN   

                total_reads_RNA_seq               TCR_derived_by_RNA_seq_tool  \
                               mean           std                        mean   
tissue                                                                          
PBMC                   6.684742e+07  2.636386e+07                  761.733179   
kidney                 8.834163e+07  3.228514e+07                    0.419664   
lymph_node             6.599892e+07           NaN                   93.077283   
melanoma               7.471916e+07  1.658280e+07                    2.853801   
small_intestine        7.272037e+07           NaN                    4.331661   

                             
                        std  
tissue                       
PBMC             849.866315  
kidney             0.383179  
lymph_node              NaN  
melanoma           3.278579  
small_intestine         NaN

TRUST4


total_reads_TCR                total_reads_tool  \
                           mean            std             mean   
tissue                                                            
PBMC               1.215376e+05  108376.531412     91349.200000   
kidney             1.019600e+04    8349.402613       452.666667   
lymph_node         3.256697e+06            NaN      9054.000000   
melanoma           9.512597e+05  544844.963860       665.333333   
small_intestine    3.047629e+06            NaN       544.000000   

                               total_reads_RNA_seq                \
                           std                mean           std   
tissue                                                             
PBMC             107795.349996        6.684742e+07  2.636386e+07   
kidney              226.780364        8.834163e+07  3.228514e+07   
lymph_node                 NaN        6.599892e+07           NaN   
melanoma            888.407846        7.471916e+07  1.658280e+07   
small_intestine            NaN        7.272037e+07           NaN   

                TCR_derived_by_RNA_seq_tool               
                                       mean          std  
tissue                                                    
PBMC                            1036.619505  1050.450769  
kidney                             6.159241     5.174758  
lymph_node                       137.184067          NaN  
melanoma                           8.780548    11.009198  
small_intestine                    7.480710          NaN

Average number of reads in different repertoire types

In [6]:
mean_repertoire_reads = reads_count.groupby("class")["total_reads_TCR","total_reads_tool","total_reads_RNA_seq","TCR_derived_by_RNA_seq_tool"].agg(["mean", "std"]) 
display(mean_repertoire_reads)
tools = ['MIXCR','IMREP','TRUST4']              
for tool in tools: 
    print(tool)
    df_tool = reads_count.loc[reads_count['tool'] == tool]
    mean_repertoire_reads = df_tool.groupby("class")["total_reads_TCR","total_reads_tool","total_reads_RNA_seq","TCR_derived_by_RNA_seq_tool"].agg(["mean", "std"]) 
    display(mean_repertoire_reads)

total_reads_TCR               total_reads_tool  \
                                  mean           std             mean   
class                                                                   
T_cell_poor_monoclonal    1.295400e+04  0.000000e+00      1590.000000   
T_cell_poor_polyclonal    9.688833e+05  8.556907e+05       204.333333   
T_cell_rich_monoclonal    1.614307e+05  1.083986e+05    113584.555556   
T_cell_rich_polyclonal    1.126698e+06  1.597935e+06      2268.111111   

                                     total_reads_RNA_seq                \
                                 std                mean           std   
class                                                                    
T_cell_poor_monoclonal   1101.385945        8.062250e+07  0.000000e+00   
T_cell_poor_polyclonal    249.095048        7.746627e+07  2.011370e+07   
T_cell_rich_monoclonal  78773.384868        8.351077e+07  1.613441e+07   
T_cell_rich_polyclonal   3185.050763        4.990124e+07  1.212788e+07   

                       TCR_derived_by_RNA_seq_tool              
                                              mean         std  
class                                                           
T_cell_poor_monoclonal                   19.721541   13.661024  
T_cell_poor_polyclonal                    2.837563    3.558914  
T_cell_rich_monoclonal                 1272.770106  654.592373  
T_cell_rich_polyclonal                   37.868287   47.668130

MIXCR


total_reads_TCR               total_reads_tool  \
                                  mean           std             mean   
class                                                                   
T_cell_poor_monoclonal    1.295400e+04           NaN      1071.000000   
T_cell_poor_polyclonal    9.688833e+05  8.812393e+05        68.916667   
T_cell_rich_monoclonal    1.614307e+05  1.251679e+05     75302.666667   
T_cell_rich_polyclonal    1.126698e+06  1.845137e+06       519.666667   

                                     total_reads_RNA_seq                \
                                 std                mean           std   
class                                                                    
T_cell_poor_monoclonal           NaN        8.062250e+07           NaN   
T_cell_poor_polyclonal     92.146581        7.746627e+07  2.071424e+07   
T_cell_rich_monoclonal  49798.098451        8.351077e+07  1.863041e+07   
T_cell_rich_polyclonal    717.476364        4.990124e+07  1.400407e+07   

                       TCR_derived_by_RNA_seq_tool              
                                              mean         std  
class                                                           
T_cell_poor_monoclonal                   13.284133         NaN  
T_cell_poor_polyclonal                    0.981210    1.331736  
T_cell_rich_monoclonal                  849.436733  366.742010  
T_cell_rich_polyclonal                    8.470190   10.390502

IMREP


total_reads_TCR               total_reads_tool  \
                                  mean           std             mean   
class                                                                   
T_cell_poor_monoclonal    1.295400e+04           NaN            844.0   
T_cell_poor_polyclonal    9.688833e+05  8.812393e+05            124.5   
T_cell_rich_monoclonal    1.614307e+05  1.251679e+05         113933.0   
T_cell_rich_polyclonal    1.126698e+06  1.845137e+06           2536.0   

                                     total_reads_RNA_seq                \
                                 std                mean           std   
class                                                                    
T_cell_poor_monoclonal           NaN        8.062250e+07           NaN   
T_cell_poor_polyclonal    127.105897        7.746627e+07  2.071424e+07   
T_cell_rich_monoclonal  91222.245516        8.351077e+07  1.863041e+07   
T_cell_rich_polyclonal   3184.980220        4.990124e+07  1.400407e+07   

                       TCR_derived_by_RNA_seq_tool              
                                              mean         std  
class                                                           
T_cell_poor_monoclonal                   10.468541         NaN  
T_cell_poor_polyclonal                    1.733860    1.737552  
T_cell_rich_monoclonal                 1258.189905  721.166385  
T_cell_rich_polyclonal                   42.391155   46.168832

TRUST4


total_reads_TCR               total_reads_tool  \
                                  mean           std             mean   
class                                                                   
T_cell_poor_monoclonal    1.295400e+04           NaN      2855.000000   
T_cell_poor_polyclonal    9.688833e+05  8.812393e+05       419.583333   
T_cell_rich_monoclonal    1.614307e+05  1.251679e+05    151518.000000   
T_cell_rich_polyclonal    1.126698e+06  1.845137e+06      3748.666667   

                                     total_reads_RNA_seq                \
                                 std                mean           std   
class                                                                    
T_cell_poor_monoclonal           NaN        8.062250e+07           NaN   
T_cell_poor_polyclonal    308.670598        7.746627e+07  2.071424e+07   
T_cell_rich_monoclonal  98299.440543        8.351077e+07  1.863041e+07   
T_cell_rich_polyclonal   4683.804152        4.990124e+07  1.400407e+07   

                       TCR_derived_by_RNA_seq_tool              
                                              mean         std  
class                                                           
T_cell_poor_monoclonal                   35.411950         NaN  
T_cell_poor_polyclonal                    5.797618    4.567406  
T_cell_rich_monoclonal                 1710.683680  708.940058  
T_cell_rich_polyclonal                   62.743517   67.780944